# CarND-Behavioral-Cloning-P3

## Read from csv file

In [25]:
import csv
import cv2
import numpy as np

def read_data(path):
    print("Start read training data from path: " + path)
    driving_log_path = path + "driving_log.csv"
    lines = []
    
    with open(driving_log_path) as csv_file:
        reader = csv.reader(csv_file)
        for line in reader:
            lines.append(line)
    print("Read from csv file done!")
    
    images = []
    measurements = []
    for i, line in enumerate(lines):
        if i == 0:
            continue
        # read in center images
        image_path = line[0]
        file_name = image_path.split("/")[-1]
        file_path = path + "IMG/"
        data_path = file_path + file_name
        image = cv2.imread(data_path)
        images.append(image)
        # read in angles
        measurement = float(line[-1])
        measurements.append(measurement)
    print("Extract center images and angle measurements from file done! Get {0} samples".format(len(images)))
    
    # Data augment
    augment_images, augment_measurements = [], []
    for image, measurement in zip(images, measurements):
        augment_images.append(image)
        augment_measurements.append(measurement)
        augment_images.append(cv2.flip(image, 1))
        augment_measurements.append(measurement * -1.0)
    print("Data augment done!")
    print()

    return augment_images, augment_measurements
    

def merge_data(path, X_train = None, y_train = None):
    custom_images, custom_measurements = read_data(path)
    if X_train is None:
        print("Create data sets...")
        X_train = np.array(custom_images)
        y_train = np.array(custom_measurements)
    else:   
        print("Concat data sets...")
        X_train = np.concatenate((X_train, np.array(custom_images)))
        y_train = np.concatenate((y_train, np.array(custom_measurements)))
    return X_train, y_train

# Read from data/
X_train, y_train = merge_data("data/")
# X_train, y_train = merge_data("custom_data/", X_train, y_train)
# X_train, y_train = merge_data("custom_data_0211/", X_train, y_train)
# X_train, y_train = merge_data("custom_data_0212/", X_train, y_train)


print("Done! Get {0} samples.".format(len(X_train)))

Start read training data from path: data/
Read from csv file done!


Extract center images and angle measurements from file done! Get 8036 samples


Data augment done!

Create data sets...


Done! Get 16072 samples.


Since the data is getting more and more. We have to change the read csv function to stream way to reduce memory consume.

In [26]:
import csv
import cv2
import numpy as np

def read_data_stream(path):
    print("Start read training data from path: " + path)
    driving_log_path = path + "driving_log.csv"
    flag = True
    
    line_num = sum(1 for line in open(driving_log_path))
    with open(driving_log_path) as csv_file:
        reader = csv.reader(csv_file)
        batch = 10000
        batch_counter = 0
        images = []
        measurements = []
        
        for i, line in enumerate(reader):
            # Ignore the first line
            if flag is True:
                flag = False
                continue
            
            # Read in center images
            image_path = line[0]
            file_name = image_path.split("/")[-1]
            file_path = path + "IMG/"
            data_path = file_path + file_name
            image = cv2.imread(data_path)
            # read in angles
            measurement = float(line[-1])
            
            images.append(image)
            measurements.append(measurement)
            
            # Data augment
            augment_image = cv2.flip(image, 1)
            augment_measurement = measurement * -1.0
            
            images.append(augment_image)
            measurements.append(augment_measurement)
            
            if i % batch == 0 or i >= line_num - 1:
                batch_counter += 1
                print("The [{0}] batch processed with batch size[{1}].".format(batch_counter, batch))
                yield images, measurements
    

def merge_data(path, X_train = None, y_train = None):
    for custom_images, custom_measurements in read_data_stream(path): 
        if X_train is None:
            X_train = np.array(custom_images)
            y_train = np.array(custom_measurements)
        else:
            X_train = np.concatenate((X_train, np.array(custom_images)))
            y_train = np.concatenate((y_train, np.array(custom_measurements)))
        
    print("Merged data size: {0}".format(X_train.shape))
    return X_train, y_train

# Read from data/
X_train, y_train = merge_data("data/")
X_train, y_train = merge_data("custom_data_0208/", X_train, y_train)
X_train, y_train = merge_data("custom_data_0211/", X_train, y_train)
X_train, y_train = merge_data("custom_data_0212/", X_train, y_train)


print("Done! Get {0} samples.".format(len(X_train)))

Start read training data from path: data/


The [1] batch processed with batch size[10000].


Start read training data from path: custom_data_0208/


The [1] batch processed with batch size[10000].


Start read training data from path: custom_data_0211/


The [1] batch processed with batch size[10000].


### Test the training data

In [13]:
print("X_train shape: " + str(X_train.shape))
print("y_train shape: " + str(y_train.shape))

X_train shape: (32898, 160, 320, 3)
y_train shape: (32898,)


## Train the model

In [12]:
# # Train model
# from keras.models import Sequential, Model
# from keras.layers import Lambda, Flatten, Dense
# 
# print("Start training the model...")
# 
# model = Sequential()
# model.add(Flatten(input_shape=(160,320,3)))
# model.add(Dense(1))
# 
# model.compile(loss='mse', optimizer='adam')
# model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=8)
# 
# model.save("model.h5")
# print("Training done, save model to model.h5")

Turns out the flow is working. But it is not working fine. To improve the model we can add preprocessing steps to the training process. 

Add the Navida power architecture to train the model.

In [13]:
# Train model
import os  

from keras.models import Sequential, Model
from keras.layers import Lambda, Flatten, Dense, Cropping2D, Convolution2D, Dropout

print("Start training the model...")

model = Sequential()
model.add(Lambda(lambda x : (x / 255.0) - 0.5, input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((70, 25), (0, 0))))
model.add(Convolution2D(24, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(36, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(48, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dropout(0.9))
model.add(Dense(50))
model.add(Dropout(0.9))
model.add(Dense(10))
model.add(Dropout(0.9))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=5)

model_name = "models/model_0212_dropout.h5"
model.save(model_name)
print("Training done, save model to {0}".format(model_name))




# Play beep alert after training done!
import pygame
import time

pygame.mixer.init()
track=pygame.mixer.music.load("/Users/dianping/sdc_workspace/alert.mp3") #可以播放.mp3 .wav等多种格式的音频文件
loop_time = 5

for i in range(loop_time):
    pygame.mixer.music.play()
    time.sleep(10)  


Start training the model...


Train on 26318 samples, validate on 6580 samples
Epoch 1/5


   32/26318 [..............................] - ETA: 858s - loss: 849.9187

   64/26318 [..............................] - ETA: 546s - loss: 870.7398

   96/26318 [..............................] - ETA: 454s - loss: 853.7891

...

26272/26318 [============================>.] - ETA: 0s - loss: 851.7187

26304/26318 [============================>.] - ETA: 0s - loss: 851.6993

26318/26318 [==============================] - 272s - loss: 851.6304 - val_loss: 799.9944


Training done, save model to models/model_0212_dropout.h5


ImportError: No module named 'pygame'

To learn from previous model. We use transfer learning here to save time and compute resource.

In [8]:
# Play beep alert after training done!
import pygame
import time

pygame.mixer.init()
track=pygame.mixer.music.load("/Users/dianping/sdc_workspace/alert.mp3") #可以播放.mp3 .wav等多种格式的音频文件
loop_time = 5

for i in range(loop_time):
    pygame.mixer.music.play()
    time.sleep(10)  


ImportError: No module named 'pygame'